In [ ]:
# !pip install ipython-autotime
%load_ext autotime

## Installing the required packages

In [ ]:
# !pip install transformers
# !pip install sentencepiece

In [ ]:
# !apt install neofetch

## Importing required libraries

In [ ]:
# !pip install torch

In [1]:
import pandas as pd
import os
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers.optimization import  Adafactor 
import time
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# !neofetch

## Loading the processed data 

In [23]:
train_df=pd.read_csv('webNLG2020_train.csv', index_col=[0])

In [24]:
train_df

,prefix,input_text,target_text
0,webNLG,"Aarhus_Airport | cityServed | ""Aarhus, Denmark""","The Aarhus is the airport of Aarhus, Denmark."
1,webNLG,"Aarhus_Airport | cityServed | ""Aarhus, Denmark""","Aarhus Airport serves the city of Aarhus, Denm..."
2,webNLG,Aarhus_Airport | cityServed | Aarhus,Aarhus airport serves the city of Aarhus.
3,webNLG,Aarhus_Airport | elevationAboveTheSeaLevel | 25.0,Aarhus Airport is 25 metres above sea level.
4,webNLG,Aarhus_Airport | elevationAboveTheSeaLevel | 25.0,Aarhus airport is at an elevation of 25 metres...
...,...,...,...
35193,webNLG,Visvesvaraya_Technological_University | city |...,The Acharya Institute of Technology's campus i...
35194,webNLG,Visvesvaraya_Technological_University | city |...,Acharya Institute of Technology in India is af...
35195,webNLG,Visvesvaraya_Technological_University | city |...,"In Soldevanahalli, Acharya Dr. Sarvapalli Radh..."
35196,webNLG,Visvesvaraya_Technological_University | city |...,The Acharya Institute of Technology was establ...


In [25]:
train_df=train_df.iloc[  :35000,:]

In [26]:
train_df

,prefix,input_text,target_text
0,webNLG,"Aarhus_Airport | cityServed | ""Aarhus, Denmark""","The Aarhus is the airport of Aarhus, Denmark."
1,webNLG,"Aarhus_Airport | cityServed | ""Aarhus, Denmark""","Aarhus Airport serves the city of Aarhus, Denm..."
2,webNLG,Aarhus_Airport | cityServed | Aarhus,Aarhus airport serves the city of Aarhus.
3,webNLG,Aarhus_Airport | elevationAboveTheSeaLevel | 25.0,Aarhus Airport is 25 metres above sea level.
4,webNLG,Aarhus_Airport | elevationAboveTheSeaLevel | 25.0,Aarhus airport is at an elevation of 25 metres...
...,...,...,...
34995,webNLG,Azerbaijan | capital | Baku && Baku_Turkish_Ma...,Huseyin Butuner and Hilmi Guner designed the r...
34996,webNLG,Azerbaijan | capital | Baku && Baku_Turkish_Ma...,Baku is the capital of Azerbaijan which is led...
34997,webNLG,Azerbaijan | capital | Baku && Baku_Turkish_Ma...,The Baku Turkish Martyrs' Memorial was designe...
34998,webNLG,Azerbaijan | capital | Baku && Baku_Turkish_Ma...,The Baku Turkish Martyrs' Memorial was designe...


In [27]:
train_df=train_df.sample(frac = 1)

In [28]:
train_df

,prefix,input_text,target_text
34103,webNLG,United_States | ethnicGroup | Asian_Americans ...,A Fortress of Grey Ice is a book from the Unit...
10406,webNLG,"Atlantic_City,_New_Jersey | country | United_S...",The United States includes the ethnic group of...
3433,webNLG,"Angola,_Indiana | country | United_States","Angola, Indiana is in the United States."
15990,webNLG,Akeem_Ayers | formerTeam | Los_Angeles_Rams &&...,"Akeem Ayers, who previously and currently play..."
25013,webNLG,"Antioch,_California | populationTotal | 102372...","The population of Antioch, California is 10237..."
...,...,...,...
614,webNLG,Pakistan | leader | Mamnoon_Hussain,Mamnoon Hussain is the leader of Pakistan.
7801,webNLG,Antwerp_International_Airport | owner | Flemis...,"Antwerp International Airport, serving the cit..."
18765,webNLG,Bakewell_pudding | region | Derbyshire_Dales &...,Bakewell pudding (a variation of which is Bake...
1140,webNLG,Andra_(singer) | associatedBand/associatedMusi...,Andra is a singer associated with Andreea Balan.


In [ ]:
batch_size=8
num_of_batches=len(train_df)/batch_size
num_of_epochs=4

In [ ]:
num_of_batches=int(num_of_batches)

In [ ]:
# !pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.9-cp37-cp37m-linux_x86_64.whl

In [ ]:
# #tpu

# # imports the torch_xla package
# import torch_xla
# import torch_xla.core.xla_model as xm

# dev = xm.xla_device(devkind='TPU')
# t1 = torch.ones(3, 3, device = dev)
# print(t1)

Checking for the GPU availability

In [ ]:
# !pip install torch==1.9.0+cu111 torchvision==0.10.0+cu111 torchaudio===0.9.0 -f https://download.pytorch.org/whl/torch_stable.html

In [2]:
import torch
import pycuda.driver as cuda
cuda.init()

torch.cuda.current_device()
cuda.Device(0).name()

'NVIDIA GeForce RTX 2080 SUPER'

In [ ]:
# !pip install pycuda

In [3]:
dev = torch.device("cuda") 

In [4]:
torch.cuda.is_available()

True

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda") 
    print("Running on the GPU")
else:
    dev = torch.device("cpu")
    print("Running on the CPU")

## Loading the pretrained model and tokenizer

In [5]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)
#moving the model to device(GPU/CPU)
model.to(dev)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

## Initializing the Adafactor optimizer with parameter values suggested for t5

In [ ]:

optimizer = Adafactor(
    model.parameters(),
    lr=1e-3,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False
)

  






In [ ]:

from IPython.display import HTML, display

def progress(loss,value, max=100):
    return HTML(""" Batch loss :{loss}
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(loss=loss,value=value, max=max))

In [ ]:
num_of_epochs=1

## Training the model

In [ ]:
#Sets the module in training mode
model.train()

loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
  print('Running epoch: {}'.format(epoch))
  
  running_loss=0

  out = display(progress(1, num_of_batches+1), display_id=True)
  for i in range(num_of_batches):
    inputbatch=[]
    labelbatch=[]
    new_df=train_df[i*batch_size:i*batch_size+batch_size]
    for indx,row in new_df.iterrows():
      input = 'WebNLG: '+row['input_text']+'</s>' 
      labels = row['target_text']+'</s>'   
      inputbatch.append(input)
      labelbatch.append(labels)
    inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=400,return_tensors='pt')["input_ids"]
    labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=400,return_tensors="pt") ["input_ids"]
    inputbatch=inputbatch.to(dev)
    labelbatch=labelbatch.to(dev)

    # clear out the gradients of all Variables 
    optimizer.zero_grad()

    # Forward propogation
    outputs = model(input_ids=inputbatch, labels=labelbatch)
    loss = outputs.loss
    loss_num=loss.item()
    logits = outputs.logits
    running_loss+=loss_num
    if i%10 ==0:      
      loss_per_10_steps.append(loss_num)
    out.update(progress(loss_num,i, num_of_batches+1))

    # calculating the gradients
    loss.backward()

    #updating the params
    optimizer.step()
    
  running_loss=running_loss/int(num_of_batches)
  print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))
  

## Plotting the loss over time

In [ ]:
import matplotlib.pyplot as plt
   
steps = [i*100 for i in range(len(loss_per_10_steps))]
  
plt.plot(steps, loss_per_10_steps)
plt.title('Loss')
plt.xlabel('Steps')
plt.ylabel('Loss')
plt.show()

## Testing the model

In [ ]:
model.eval()
input_ids = tokenizer.encode("WebNLG: sidharth | hometown | Delhi && sidharth | play |  football </s>", return_tensors="pt")  # Batch size 1
input_ids=input_ids.to(dev)
outputs = model.generate(input_ids)
tokenizer.decode(outputs[0])

Before testing the model further, lets learn how to serialize it and load from the path

## Serializing the trained model

In [ ]:
torch.save(model.state_dict(),'pytorch_model_4epoch_1809.bin')


## Downloading the config file

In [ ]:
# !wget https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json

## Loading the trained model from the path

In [6]:
model = T5ForConditionalGeneration.from_pretrained('pytorch_model_4epoch_1809.bin', return_dict=True,config='t5-base-config.json')


In [7]:
model

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

## The Inference function

In [8]:
# tokenizer = T5Tokenizer.from_pretrained('t5-base')

def generate(text):
  model.eval()
  input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), return_tensors="pt")  # Batch size 1
  # input_ids.to(dev)
  s = time.time()
  outputs = model.generate(input_ids)
  gen_text=tokenizer.decode(outputs[0]).replace('<pad>','').replace('</s>','')
  elapsed = time.time() - s
  print('Generated in {} seconds'.format(str(elapsed)[:4]))

  
  return gen_text

# Now, Lets test it out !

In [9]:
generate(' Russia | leader | Putin')

Generated in 9.93 seconds


' Putin is the leader of Russia.'

In [10]:
generate('Sidhath | profession | Doctor  && Sidharth | home_town |  Bombay')

Generated in 0.59 seconds


' Sidhath, a doctor, is in the same town as Bombay.'

In [11]:
generate('Nie_Haisheng | birthDate | 1964-10-13  && Nie_Haisheng | occupation | Fighter_pilot ')

Generated in 0.60 seconds


' Born on 1963-10-13, fighter pilot, Nie Haisheng, was a fighter pilot'

In [12]:
generate('Bananaman | creator | Steve_Bright &&  Bananaman | broadcastedBy | BBC')

Generated in 0.61 seconds


' Bananaman, which was created by Steve Bright, was broadcasted by the BBC'

In [13]:
generate('Bananaman | lastAired | "1986-04-15" && Bananaman | creator | Steve_Bright')

Generated in 0.60 seconds


' Bananaman, created by Steve Bright, last aired on April 15th 1986'

In [14]:
generate('Alan_B._Miller_Hall | currentTenants | Mason_School_of_Business && Alan_B._Miller_Hall | location | Williamsburg,_Virginia')

Generated in 0.61 seconds


' Alan B. Miller Hall is located in Williamsburg, Virginia and the Mason School of Business are'

## Results after quantization

In [15]:
import torch.quantization
import torch.nn as nn

quantized_model = torch.quantization.quantize_dynamic(
    model, {nn.Linear,nn.Dropout,nn.LayerNorm}, dtype=torch.qint8
)


Lets check the difference in size of the model

In [16]:
def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

print_size_of_model(model)
print_size_of_model(quantized_model)

Size (MB): 891.727295
Size (MB): 322.016761


In [17]:
def quant_generate(text):
  quantized_model.eval()
  input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), return_tensors="pt")  # Batch size 1
  # input_ids.to(dev)
  s = time.time()
  outputs = quantized_model.generate(input_ids)
  gen_text=tokenizer.decode(outputs[0]).replace('<pad>','').replace('</s>','')
  elapsed = time.time() - s
  print('Generated in {} seconds'.format(str(elapsed)[:4]))

  
  return gen_text

Now lets check the difference in inference time

In [18]:
quant_generate('Facebook | CEO | Mark  && Facebook | number Of Employees | 52000 ')

Generated in 0.97 seconds


' The CEO of the company, Fakebook, has 52000 employees.'

In [19]:
generate('Facebook | CEO | Mark  && Facebook | number Of Employees | 52000 ')

Generated in 0.59 seconds


' Mark is the CEO of the company, Fakebook, which has 52000 employees.'

In [21]:
quant_generate('Breakdown | CEO | Facebook')

Generated in 0.77 seconds


' The CEO of Breakdown is FB.'

In [22]:
generate('Breakdown | CEO | Facebook')

Generated in 0.42 seconds


' The CEO of Breakdown is called Fake.'